In [1]:
import os

# Replace these macros accordingly
ROOT = '/home/vincliu/vocoder/NeMo'
os.chdir(ROOT)

CHECKPOINT = os.path.join('nemo_experiments', 'SqueezeWave', '2020-08-26_13-36-50', 'checkpoints', 'SqueezeWave--last.ckpt')
CONFIG = os.path.join('examples', 'tts', 'conf', 'squeezewave.yaml')
DATA_MANIFEST = os.path.join('data', 'nvidia_ljspeech_test.json')
OUT_FOLDER = os.path.join('samples')
if not os.path.isdir(OUT_FOLDER):
    os.makedirs(OUT_FOLDER)

N_SAMPLES = 10
MAX_WAV_VALUE = 32768.
DENOISER_STRENGTH = 0.1

In [2]:
import torch
from omegaconf import OmegaConf

from nemo.collections.tts.modules.squeezewave import OperationMode
from nemo.collections.tts.models.squeezewave import SqueezeWaveModel

# Remove all weight norm reparameterizations
def remove_weightnorm(model):
    squeezewave = model.squeezewave
    assert hasattr(squeezewave, 'wavenet')
    for i in range(len(squeezewave.wavenet)):
        wavenet = squeezewave.wavenet[i]
        wavenet.start = torch.nn.utils.remove_weight_norm(wavenet.start)
        wavenet.cond_layer = torch.nn.utils.remove_weight_norm(wavenet.cond_layer)
        for j in range(wavenet.n_layers):
            res_skip_layer = wavenet.res_skip_layers[j]
            res_skip_layer = torch.nn.utils.remove_weight_norm(res_skip_layer)
    return model

# Load model with its config
def load_squeezewave_model(cfg):
    with open(cfg) as f:
        config = OmegaConf.load(f)
    del config.model['train_ds']
    del config.model['validation_ds']
    del config.model['optim']
    model = SqueezeWaveModel(cfg=config['model'])
    print('Number of trainable parameters:', model.num_weights)
    return model

# Load weights and prepare model for inference
model = load_squeezewave_model(CONFIG)
checkpoint = torch.load(CHECKPOINT)['state_dict']
model.load_state_dict(checkpoint, strict=True)
model = remove_weightnorm(model)
model = model.eval()
model.mode = OperationMode.validation
model.squeezewave.mode = OperationMode.validation

[NeMo W 2020-08-26 16:27:18 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToCharDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-08-26 16:27:18 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToBPEDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-08-26 16:27:18 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioLabelDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-08-26 16:27:18 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text._TarredAudioToTextDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-08-26 16:27:18 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.TarredAudioToCharDataset'> is experimental,

In [3]:
from torch.utils.data import DataLoader

from nemo.collections.tts.losses.waveglowloss import WaveGlowLoss
from nemo.collections.tts.data.datalayers import AudioDataset

dataset = AudioDataset(DATA_MANIFEST, n_segments=-1, truncate_to=4096)
dataloader = DataLoader(
    dataset,
    1,
    shuffle=False,
    pin_memory=True,
    drop_last=False,
    num_workers=4,
)

loss_fn = WaveGlowLoss()

[NeMo I 2020-08-26 16:27:28 collections:171] Dataset loaded with 500 files totalling 0.91 hours
[NeMo I 2020-08-26 16:27:28 collections:172] 0 files were filtered totalling 0.00 hours


In [4]:
from nemo.collections.asr.parts.features import STFTExactPad

class Denoiser(torch.nn.Module):
    def __init__(self, model, n_mel=80, filter_length=1024, hop_length=512, win_length=None, window='hann'):
        super().__init__()

        self.stft = STFTExactPad(
            filter_length=filter_length, hop_length=hop_length, win_length=win_length, window=window,
        ).to(model.device)

        with torch.no_grad():
            mel_input = torch.zeros((n_mel, 88)).to(model.device)
            bias_audio = model.convert_spectrogram_to_audio(mel_input, sigma=0.0)
            bias_spec, _ = self.stft.transform(bias_audio.unsqueeze(0))
            self.bias_spec = bias_spec[:, :, 0][:, :, None]

        model.mode = OperationMode.validation
        model.squeezewave.mode = OperationMode.validation

    def forward(self, audio, strength=0.1):
        audio_spec, audio_angles = self.stft.transform(audio)
        audio_spec_denoised = audio_spec - self.bias_spec * strength
        audio_spec_denoised = torch.clamp(audio_spec_denoised, 0.0)
        audio_denoised = self.stft.inverse(audio_spec_denoised, audio_angles)
        return audio_denoised

if DENOISER_STRENGTH > 0:
    denoiser = Denoiser(model, n_mel=80, filter_length=1024, hop_length=512, win_length=None, window='hann')

In [5]:
import soundfile as sf

from nemo.collections.asr.parts.features import FilterbankFeatures

for i, (audio, audio_len) in enumerate(dataloader):
    if i == N_SAMPLES: break
    # Get loss and audio reconstruction
    with torch.no_grad():
        z, log_s_list, log_det_W_list, audio_pred, *_ = model(audio=audio, audio_len=audio_len, run_inverse=True)
        loss = loss_fn(z=z, log_s_list=log_s_list, log_det_W_list=log_det_W_list, sigma=model.sigma)

    filename = 'audio_{}.wav'.format(i)
    print('[{}] loss: {}'.format(filename, loss.detach().cpu().numpy()))

    if DENOISER_STRENGTH > 0:
        audio_pred = denoiser(audio_pred, strength=DENOISER_STRENGTH)

    audio_pred = (audio_pred.detach().cpu().numpy().squeeze() * MAX_WAV_VALUE).astype('int16')
    sf.write(os.path.join(OUT_FOLDER, 'synth_' + filename), audio_pred, samplerate=22050)

    audio = (audio.detach().cpu().numpy().squeeze() * MAX_WAV_VALUE).astype('int16')
    sf.write(os.path.join(OUT_FOLDER, 'orig_' + filename), audio, samplerate=22050)

[audio_0.wav] loss: -5.559649467468262
[audio_1.wav] loss: -5.758926868438721
[audio_2.wav] loss: -5.894212245941162
[audio_3.wav] loss: -6.185265064239502
[audio_4.wav] loss: -5.8412370681762695
[audio_5.wav] loss: -5.563751697540283
[audio_6.wav] loss: -5.731264114379883
[audio_7.wav] loss: -5.682397842407227
[audio_8.wav] loss: -5.571674823760986
[audio_9.wav] loss: -5.983452320098877
